In [1]:
import warnings
warnings.filterwarnings('ignore')
import spectral.io.envi as envi
import scipy.io as sio
import matplotlib.pyplot as plt
import math
import random
import numpy as np
import scipy as sp
from scipy import linalg
import array
import sys
sys.path.append('../source/endmembers')
import evolutionary
import helpers.auxiliar as auxiliar
import pandas as pd
import seaborn as sns;
sns.color_palette("pastel",7);
from functools import reduce
envi_data = envi.open('../data/real/cuprite/cuprite.hdr')
envi = envi_data[:,:,:]
cuprite_path = '../data/real/cuprite/cuprite.mat'
data = sio.loadmat(cuprite_path)
gt_cuprite_path = '../data/real/cuprite/cuprite.gt'
gt = sio.loadmat(gt_cuprite_path)
M_gt = gt['M']
aux = M_gt[data['SlectBands']]
M_gt = aux.reshape((aux.shape[0],aux.shape[2]))[1:,:]
Y = data['Y'][1:,:]
number_rows = data['nRow'][0][0]
number_columns = data['nCol'][0][0]
number_pixels = int(number_rows)*int(number_columns)
number_endmembers = M_gt.shape[1]
number_bands = Y.shape[0]
dimensions = [number_rows, number_columns, number_bands]
r = 4
c = 3

In [2]:
selected_bands = data['SlectBands'].T[0]
gtreal_cuprite_path = '../data/real/cuprite/cuprite.gtreal'
gtreal = sio.loadmat(gtreal_cuprite_path)
ground_truth = gtreal['M'].T
null_bands = list(set(list(range(1,len(ground_truth)))) - set(selected_bands))
ground_truth[null_bands,:] = None

![alt text](tetracorder.png "Title")

# Ground Truth USGS Cuprite Tetracorde

In [3]:
auxiliar.plot_endmembers(ground_truth,34,'r',4,3)

TypeError: 'int' object is not subscriptable

In [ ]:
# envi_data = envi.open('../data/real/jasper/jasper.hdr')
# jasper_envi = envi_data[:,:,:]
# jasper_path = '../data/real/jasper/jasper.mat'
# data = sio.loadmat(jasper_path)
# gt_jasper_path = '../data/real/jasper/jasper.gt'
# gt = sio.loadmat(gt_jasper_path)
# M_gt = gt['M']
# Y = data['Y'][1:,:]
# number_rows = data['nRow'][0][0]
# number_columns = data['nCol'][0][0]
# number_pixels = int(number_rows)*int(number_columns)
# number_endmembers = M_gt.shape[1]
# number_bands = Y.shape[0]
# dimensions = [number_rows, number_columns, number_bands]

# Ground Truth Cuprite

In [ ]:
auxiliar.plot_endmembers(M_gt,number_endmembers,'r',r,c)

# GAEE

In [ ]:
[M, duration, generations_fitness, generations_population, number_generations] = evolutionary.GAEE(Y, dimensions, number_endmembers)
print('running time:',duration)
auxiliar.plot_endmembers(M,number_endmembers,color='b', r=r , c=c)
auxiliar.plot_convergence(generations_fitness, number_generations)
auxiliar.plot_genes(envi, generations_population, number_rows, band=30, number_generations=number_generations, r=2, c=2)

# GAEE-IVFm

In [ ]:
[M, duration, generations_fitness, generations_population, number_generations] = evolutionary.GAEE_IVFm(Y, dimensions, number_endmembers)
print('running time:',duration)
auxiliar.plot_endmembers(M,number_endmembers,color='b', r=r , c=c)
auxiliar.plot_convergence(generations_fitness, number_generations)
auxiliar.plot_genes(envi, generations_population, number_rows, band=30, number_generations=number_generations, r=2, c=2)

# GAEEII

In [ ]:
[M, other] = evolutionary.GAEEII(Y, dimensions, number_endmembers)
print('running time:',duration)
auxiliar.plot_endmembers(M,number_endmembers,color='b', r=r , c=c)
auxiliar.plot_dual_convergence(generations_fitness_1,generations_fitness_2,number_generations)
auxiliar.plot_genes(envi, generations_population, number_rows, band=30, number_generations=number_generations, r=2, c=2)

# GAEEII-IVFm

In [ ]:
[M, duration, generations_fitness_1, generations_fitness_2, generations_population, number_generations] = evolutionary.GAEEII_IVFm(Y, dimensions, number_endmembers)
print('running time:',duration)
auxiliar.plot_endmembers(M,number_endmembers,color='b', r=r , c=c)
auxiliar.plot_dual_convergence(generations_fitness_1,generations_fitness_2,number_generations)
auxiliar.plot_genes(envi, generations_population, number_rows, band=30, number_generations=number_generations, r=2, c=2)
auxiliar.check_endmember_extraction(M, M_gt, 0.005)

# COMPARER WITH TETACORDER

In [ ]:
[M,
 duration,
 generations_fitness_1,
 generations_fitness_2,
 generations_population,
 number_generations] = evolutionary.GAEEII_IVFm(Y,
                                                dimensions,
                                                number_endmembers)

new_M = np.zeros((ground_truth.shape[0],number_endmembers))
new_M[null_bands,:] = None
new_M[selected_bands[1:],:] = M
print('running time:',duration)
# auxiliar.plot_endmembers(M,number_endmembers,color='b', r=r , c=c)
# auxiliar.plot_dual_convergence(generations_fitness_1,generations_fitness_2,number_generations)
# auxiliar.plot_genes(envi, generations_population, number_rows, band=30, number_generations=number_generations, r=2, c=2)
auxiliar.check_endmember_extraction(new_M, ground_truth, 0.005)